# Pobranie danych oraz wybór dla wskazanych klas do dalszej analizy


---



In [ ]:
import pickle
import glob
import numpy as np
import pandas as pd

In [ ]:
def unpickle(file):
    all_dict = []
    meta_dict = {}
    for f in glob.glob(file):
      if f.endswith('.meta')!=True:
        print(f)
        with open(f, 'rb') as fo:
          dict = pickle.load(fo, encoding='bytes')
        all_dict.append(dict)
      else:
          with open(f, 'rb') as fo:
           meta_dict = pickle.load(fo, encoding='bytes')
    return all_dict, meta_dict

def extraction_data(data,index_my_categories):
  pic_data = data[b"data"]
  labels = data[b"labels"]
  pictures =  np.concatenate([pic_data.loc[i] for i in range(0,pic_data.shape[0])],axis=0)
  index =  np.concatenate([labels.loc[i] for i in range(0,labels.shape[0]) ],axis=0)
  return extraction_categories(index_my_categories, index, pictures)

def extraction_categories(index, index_all, pic_data):
   data = np.array([ pic_data[i] for i in range(len(index_all)) if index_all[i] in index ])
   label = np.array([ i for i in index_all if i in index ])
   return data, label

def extraction_meta(meta_d,key,type_img):
  data = meta[key]
  data = [i.decode("utf-8") for i in data]
  index_list = [data.index(i) for i in type_img]
  return index_list


In [ ]:
file = "/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/*"
data, meta = unpickle(file)

/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_3
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_2
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_5
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_1
/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/train/data_batch_4


In [ ]:
index = extraction_meta(meta,b'label_names',["airplane", "cat", "truck"])
X_train_data,y_train_data = extraction_data(pd.DataFrame(data),index)

#Przetwarzanie wstępne


---



In [ ]:
import random
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

In [ ]:
def preproces(pictures):
    pictures_copy = pictures.copy()
    scaler =  MinMaxScaler(feature_range=(0, 1))
    stand_pic = scaler.fit_transform(pictures_copy)
    return stand_pic

In [ ]:
pca = PCA(n_components=40)
x_train_pre = preproces(X_train_data)
pca = pca.fit(x_train_pre)

In [ ]:
x_train = pca.transform(x_train_pre)

#Klasyfikacja danych


---




In [ ]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=3072, solver='adam',activation='relu',max_iter=500,random_state=1,early_stopping=True)

In [ ]:
model.fit(x_train, y_train_data)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=3072, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

#Walidacja modelu (dla danych testowych) wraz z wyznaczeniem i wydrukiem macierzy pomyłek oraz wartości F1


---



In [ ]:
from sklearn.metrics import f1_score, classification_report,confusion_matrix
from sklearn import metrics

In [ ]:
file = "/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/test/*"
data, meta = unpickle(file)
X_test_data,y_test = extraction_data(pd.DataFrame(data),index)

X_test_pre= preproces(X_test_data)

X_test = pca.transform(X_test_pre)

/content/drive/MyDrive/uczelnia/UM/Wykład/cifar-10-batches-py/test/test_batch


In [ ]:
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred),"\n")
print("Confusion matrix\n",confusion_matrix(y_test, y_pred, labels=[0,3,9]))
print("\nF1-score",f1_score(y_test, y_pred, average='weighted'))
print("Raport klasyfikacji",classification_report(y_test, y_pred, target_names=["0","3","9"]))

Accuracy: 0.8223333333333334 

Confusion matrix
 [[851  71  78]
 [ 89 806 105]
 [ 89 101 810]]

F1-score 0.8222154778398617
Raport klasyfikacji               precision    recall  f1-score   support

           0       0.83      0.85      0.84      1000
           3       0.82      0.81      0.81      1000
           9       0.82      0.81      0.81      1000

    accuracy                           0.82      3000
   macro avg       0.82      0.82      0.82      3000
weighted avg       0.82      0.82      0.82      3000

